In [ ]:
#sep-10
# ============================
# NeurIPS 2025: ChemBERTa-77M-MTR + PPDCPoA + RDKit Gated Fusion
# ============================
import sys, subprocess, os
wheel_dir="/kaggle/input/rdkit-wheels/rdkit_wheels"

subprocess.check_call([
    sys.executable, "-m", "pip", "install", 
    "--no-index", "--find-links=" + wheel_dir, 
    "rdkit-pypi==2022.9.5", 
    "xgboost", "lightgbm", "catboost",
    "numpy", "pandas", "scikit-learn", "tqdm", "optuna",
    "transformers"
])

import random
import numpy as np
import pandas as pd
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error

from rdkit import Chem
from rdkit.Chem import Descriptors
from transformers import AutoTokenizer, AutoModel

# ============================
# 0. Reproducibility & device
# ============================
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
MAX_LEN = 128
BATCH_SIZE = 16
EPOCHS = 100

# Best Optuna params for RNN+RDKit
LR = 0.00095
BEST_EMBED_DIM = 128
BEST_RNN_HIDDEN = 64
BEST_RDKIT_HIDDEN = 128
BEST_DROPOUT = 0.164

# ============================
# 1. Load Data
# ============================
train = pd.read_csv("/kaggle/input/neurips-open-polymer-prediction-2025/train.csv")
test  = pd.read_csv("/kaggle/input/neurips-open-polymer-prediction-2025/test.csv")

supp_tg  = pd.read_csv("/kaggle/input/neurips-open-polymer-prediction-2025/train_supplement/dataset3.csv")
supp_ffv = pd.read_csv("/kaggle/input/neurips-open-polymer-prediction-2025/train_supplement/dataset4.csv")
supp_tc  = pd.read_csv("/kaggle/input/neurips-open-polymer-prediction-2025/train_supplement/dataset1.csv")

targets = ["Tg", "FFV", "Tc", "Density", "Rg"]

for col in targets:
    if col != "Tg": supp_tg[col] = np.nan
for col in targets:
    if col != "FFV": supp_ffv[col] = np.nan
supp_tc = supp_tc.rename(columns={"TC_mean": "Tc"})
for col in targets:
    if col != "Tc": supp_tc[col] = np.nan

train_full = pd.concat([train, supp_tg, supp_ffv, supp_tc], ignore_index=True)

# ============================
# 2. Tokenizers & SMILES Encoding
# ============================
# ChemBERTa-MTR
chemberta_path = "/kaggle/input/chemberta-77m-mtr/pytorch/default/1/ChemBERTa-77M-MTR"
chem_tokenizer = AutoTokenizer.from_pretrained(chemberta_path)

# RNN SMILES tokenizer
all_smiles = pd.concat([train_full["SMILES"], test["SMILES"]])
vocab = sorted(set("".join(all_smiles.values)))
stoi = {ch: i+1 for i,ch in enumerate(vocab)}  # 0 reserved
def encode_smiles(s, max_len=MAX_LEN):
    tokens = [stoi.get(ch,0) for ch in s[:max_len]]
    return tokens + [0]*(max_len-len(tokens))

train_full["encoded"] = train_full["SMILES"].apply(encode_smiles)
test["encoded"] = test["SMILES"].apply(encode_smiles)

# ============================
# 3. RDKit Descriptors
# ============================
def calc_rdkit_feats(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return np.zeros(5)
    return np.array([
        Descriptors.MolWt(mol),
        Descriptors.MolLogP(mol),
        Descriptors.TPSA(mol),
        Descriptors.NumHDonors(mol),
        Descriptors.NumHAcceptors(mol)
    ])

train_rdkit = np.vstack(train_full["SMILES"].apply(calc_rdkit_feats).values)
test_rdkit  = np.vstack(test["SMILES"].apply(calc_rdkit_feats).values)

scaler_rdkit = StandardScaler()
train_rdkit = scaler_rdkit.fit_transform(train_rdkit)
test_rdkit  = scaler_rdkit.transform(test_rdkit)

# ============================
# 4. Scale Targets
# ============================
scalers = {}
for col in targets:
    scaler = StandardScaler()
    vals = train_full[col].dropna().values.reshape(-1,1)
    scaler.fit(vals)
    scalers[col] = scaler
    train_full[col] = train_full[col].apply(
        lambda v: scaler.transform([[v]])[0][0] if pd.notna(v) else np.nan
    )

# ============================
# 5. Dataset
# ============================
class FusionDataset(Dataset):
    def __init__(self, df, rdkit_feats, targets=None):
        self.smiles_rnn = np.stack(df["encoded"].values)
        self.smiles_bert = df["SMILES"].tolist()
        self.rd = rdkit_feats
        self.targets = df[targets].values if targets is not None else None

    def __len__(self): return len(self.smiles_rnn)

    def __getitem__(self, idx):
        # RNN input
        x_rnn = torch.tensor(self.smiles_rnn[idx], dtype=torch.long)
        # RDKit features
        rd = torch.tensor(self.rd[idx], dtype=torch.float)
        # ChemBERTa encoding
        enc = chem_tokenizer(
            self.smiles_bert[idx],
            truncation=True,
            padding="max_length",
            max_length=MAX_LEN,
            return_tensors="pt"
        )
        input_ids = enc["input_ids"].squeeze(0)
        attention_mask = enc["attention_mask"].squeeze(0)

        if self.targets is not None:
            y = torch.tensor(self.targets[idx], dtype=torch.float)
            return x_rnn, rd, input_ids, attention_mask, y
        return x_rnn, rd, input_ids, attention_mask

train_dataset = FusionDataset(train_full, train_rdkit, targets=targets)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_dataset = FusionDataset(test, test_rdkit, targets=None)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# ============================
# 6. Model (ChemBERTa-MTR + RNN + RDKit Gated Fusion)
# ============================
class FusionModel(nn.Module):
    def __init__(self, vocab_size, rdkit_dim, embed_dim, rnn_hidden, rdkit_hidden, dropout, num_targets=5):
        super().__init__()
        # ChemBERTa-MTR
        self.bert = AutoModel.from_pretrained(chemberta_path)

        # SMILES RNN
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
        self.conv = nn.Conv1d(embed_dim, embed_dim, kernel_size=5, padding=2)
        self.gru = nn.GRU(embed_dim, rnn_hidden, batch_first=True, bidirectional=True)
        self.attention = nn.Linear(rnn_hidden*2,1)

        # RDKit
        self.rdkit_mlp = nn.Sequential(
            nn.Linear(rdkit_dim, rdkit_hidden),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(rdkit_hidden, rdkit_hidden),
            nn.ReLU()
        )
        self.rd_proj = nn.Linear(rdkit_hidden, rnn_hidden*2)

        # Gate fusion
        self.gate_c = nn.Linear(rnn_hidden*2 + self.bert.config.hidden_size, 128, bias=False)
        self.gate_r = nn.Linear(rdkit_hidden, 128, bias=False)
        self.gate_out = nn.Linear(128, rnn_hidden*2)
        nn.init.constant_(self.gate_out.bias, -1.0)

        # Final head
        self.fc = nn.Sequential(
            nn.Linear(rnn_hidden*2, 128),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(128, num_targets)
        )

        self.rnn_hidden = rnn_hidden

    def forward(self, x_rnn, rd, input_ids, attention_mask):
        # RNN path
        x = self.embedding(x_rnn)
        x = x.permute(0,2,1)
        x = torch.relu(self.conv(x))
        x = x.permute(0,2,1)
        out, _ = self.gru(x)
        attn = torch.softmax(self.attention(out), dim=1)
        context_rnn = torch.sum(attn*out, dim=1)

        # ChemBERTa path
        bert_out = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        context_bert = bert_out.last_hidden_state[:,0,:]

        # Concatenate RNN + BERT
        context = torch.cat([context_rnn, context_bert], dim=1)

        # RDKit path
        rd_repr = self.rdkit_mlp(rd)
        rd_proj = self.rd_proj(rd_repr)

        # Gating
        g = F.relu(self.gate_c(context) + self.gate_r(rd_repr))
        gate = torch.sigmoid(self.gate_out(g))

        fused = context_rnn * (1.0 - gate) + rd_proj * gate
        preds = self.fc(fused)
        return preds

model = FusionModel(
    vocab_size=len(stoi)+1,
    rdkit_dim=train_rdkit.shape[1],
    embed_dim=BEST_EMBED_DIM,
    rnn_hidden=BEST_RNN_HIDDEN,
    rdkit_hidden=BEST_RDKIT_HIDDEN,
    dropout=BEST_DROPOUT,
    num_targets=len(targets)
).to(DEVICE)

# ============================
# 7. Training
# ============================
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

def masked_mse_loss(preds, labels):
    mask = ~torch.isnan(labels)
    return ((preds - labels)[mask]**2).mean()

best_mae = float("inf")
best_model_path = "best_model.pt"

for epoch in range(EPOCHS):
    model.train()
    total_loss = 0
    for X, rd, input_ids, attention_mask, y in tqdm(train_loader, desc=f"Epoch {epoch+1} [Train]"):
        X, rd, input_ids, attention_mask, y = X.to(DEVICE), rd.to(DEVICE), input_ids.to(DEVICE), attention_mask.to(DEVICE), y.to(DEVICE)
        optimizer.zero_grad()
        preds = model(X, rd, input_ids, attention_mask)
        loss = masked_mse_loss(preds, y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    avg_loss = total_loss/len(train_loader)

    # Evaluation
    model.eval()
    all_preds, all_true = [], []
    with torch.no_grad():
        for X, rd, input_ids, attention_mask, y in DataLoader(train_dataset, batch_size=BATCH_SIZE):
            X, rd, input_ids, attention_mask = X.to(DEVICE), rd.to(DEVICE), input_ids.to(DEVICE), attention_mask.to(DEVICE)
            preds = model(X, rd, input_ids, attention_mask).cpu().numpy()
            all_preds.append(preds)
            all_true.append(y.numpy())
    all_preds = np.vstack(all_preds)
    all_true = np.vstack(all_true)

    maes = []
    for i,col in enumerate(targets):
        mask = ~np.isnan(all_true[:,i])
        if mask.sum()==0: continue
        true_vals = scalers[col].inverse_transform(all_true[mask,i].reshape(-1,1)).flatten()
        pred_vals = scalers[col].inverse_transform(all_preds[mask,i].reshape(-1,1)).flatten()
        mae = mean_absolute_error(true_vals, pred_vals)
        maes.append(mae)
        print(f"Epoch {epoch+1} | {col}: MAE={mae:.4f}")
    avg_mae = np.mean(maes)
    print(f"Epoch {epoch+1} | Avg Train Loss={avg_loss:.4f} | Avg MAE={avg_mae:.4f}")

    if avg_mae < best_mae:
        best_mae = avg_mae
        torch.save(model.state_dict(), best_model_path)
        print(f"✅ Best model saved (epoch {epoch+1}, MAE={avg_mae:.4f})")

# ============================
# 8. Inference
# ============================
print("\nLoading best model...")
model.load_state_dict(torch.load(best_model_path))
model.eval()

preds_list = []
with torch.no_grad():
    for X, rd, input_ids, attention_mask in test_loader:
        X, rd, input_ids, attention_mask = X.to(DEVICE), rd.to(DEVICE), input_ids.to(DEVICE), attention_mask.to(DEVICE)
        out = model(X, rd, input_ids, attention_mask).cpu().numpy()
        preds_list.append(out)
preds = np.vstack(preds_list)

# Inverse transform
for i,col in enumerate(targets):
    preds[:,i] = scalers[col].inverse_transform(preds[:,i].reshape(-1,1)).flatten()

# ============================
# 9. Submission
# ============================
submission = pd.read_csv("/kaggle/input/neurips-open-polymer-prediction-2025/sample_submission.csv")
submission[targets] = preds
submission.to_csv("submission.csv", index=False)
print("✅ submission.csv saved")
